In [1]:
import os
import json
import csv
from IPython.display import display, Image
from datetime import datetime, timedelta

In [2]:
import json

instance_ids = []
json_filename = 'full_gold.json'
with open(json_filename, 'r') as jsonfile:
    lines = jsonfile.read().split("\n")[:-1]
    for line in lines:
        instance = json.loads(line)
        instance_id = instance['anchor_id'].split("_")[0] + "_" + instance['anchor_id'].split("_")[1]
        instance_ids.append(instance_id)

In [3]:
from IPython.display import display, Image

onlytweets_dir = '/home/zhaomin/Documents/saved_tweets/'
instances = []
num_context = 3
url_prefix = "http://www.cse.unt.edu/~blanco/screenshot/"
  
for event in os.listdir(onlytweets_dir):
    event_name = event.split("_")[0] + event.split("_")[1].split("-")[0]
    event_path = os.path.join(onlytweets_dir, event)
    days = os.listdir(event_path)
    for day in days:
        day_path = os.path.join(event_path, day)
        day_instances = os.listdir(day_path)
        for day_instance in day_instances:
            day_instance_path = os.path.join(day_path, day_instance)

            # load the anchor tweet
            anchor_filename = f"anchor_{day_instance}.json"
            anchor_filepath = os.path.join(day_instance_path, anchor_filename)
            with open(anchor_filepath, 'r') as anchor_jsonfile:
                anchor_tweet = json.load(anchor_jsonfile)

            location_name = " ".join([token.capitalize() for token in anchor_tweet['location'].split(" ")])

            # only use the gold instance in phase 1
            instance_id = f"{event_name}_{day_instance}"
            if instance_id not in instance_ids:
                continue

            ele = {
                'instance_id': instance_id,
                'event': f"{event_name}",
                'anchor_location': location_name,
                'anchor_timestamp': anchor_tweet['created_at'],
                'anchor_url': f"{url_prefix}{event_name}_{day_instance}_anchor_{day_instance}.png"
            }

            # load the 20 context tweets
            tweets = []
            filenames = os.listdir(day_instance_path)
            json_filenames = [x for x in filenames if x.endswith(".json") and not x.startswith("anchor")]
            for json_filename in json_filenames:
                json_filepath = os.path.join(day_instance_path, json_filename)
                with open(json_filepath, 'r') as jsonfile:
                    tweet = json.load(jsonfile)
                    tweets.append(tweet)
            context_tweets = sorted(tweets, key = lambda x: datetime.strptime(x['created_at'], '%a %b %d %H:%M:%S +0000 %Y'))

            # only keep part of the context tweets
            context_tweets = context_tweets[10-num_context:10+num_context]

            for index, context_tweet in enumerate(context_tweets):
                context_id = context_tweet['id_str']
                ele[f"context{index+(10-num_context+1)}_url"] = f"{url_prefix}{event_name}_{day_instance}_{context_id}.png"
                context_timestamp = context_tweet['created_at']
                ele[f"context{index+(10-num_context+1)}_timestamp"] = context_timestamp

            instances.append(ele)
#             break
#         break
#     break

In [10]:
import csv
import math

instances = []
csv_filepath = 'batch_6540.csv'
with open(csv_filepath, 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        instances.append(row)
        
num_instances = 500
num_batches = math.ceil(len(instances)/500)
batch_folder = 'batches'
for i in range(num_batches):
    batch_filename = f"{i+1}.csv"
    batch_filepath = os.path.join(batch_folder, batch_filename)
    if i != num_batches:
        small_instances = instances[num_instances*i:num_instances*(i+1)]
    else:
        small_instances = instances[num_instances*i:]
    with open(batch_filepath, 'w') as csvfile:
        column_names = list(small_instances[0].keys())
        writer = csv.DictWriter(csvfile, fieldnames=column_names)
        writer.writeheader()
        for instance in small_instances:
            writer.writerow(instance)

In [3]:
import csv

csv_filename = 'batch_3context.csv'
        
# generate the html file
html_filename = "display_context_anchor.html"
with open(html_filename, 'w') as htmlfile:

    with open(csv_filename, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        
        # write the start
        htmlfile.write("<!DOCTYPE html>\n<html>\n<head>\n<title>Tweet Example</title>\n</head>\n<body>\n")

        # use a big table to display the tweet example
        htmlfile.write("""<table style=\"margin-left: auto; margin-right: auto; border: 1px solid black; line-height: 1.0em; width: 1200\">\n""")
        image_css = """style=\"display: block; margin-left: auto; margin-right: auto; max-width: 450px; max-height: 650px;\""""
        
        for row in reader:

            # 3 context tweets before
            htmlfile.write("<tr>\n")

            # context8_url
            htmlfile.write("<td style=\"width:400; border-bottom: 1px solid black;\">\n")
            htmlfile.write(f"<img src=\"{row['context8_url']}\" {image_css}>\n")
            htmlfile.write("</td>\n")

            # context9_url
            htmlfile.write("<td style=\"width:400; border-bottom: 1px solid black;\">\n")
            htmlfile.write(f"<img src=\"{row['context9_url']}\" {image_css}>\n")
            htmlfile.write("</td>\n")

            # context10_url
            htmlfile.write("<td style=\"width:400; border-bottom: 1px solid black;\">\n")
            htmlfile.write(f"<img src=\"{row['context10_url']}\" {image_css}>\n")
            htmlfile.write("</td>\n")

            htmlfile.write("</tr>\n")
            
            # anchor tweet
            htmlfile.write("<tr>\n")

            # anchor_url
            htmlfile.write("<td style=\"width:400; border-bottom: 1px solid black;\">\n")
            htmlfile.write(f"<img src=\"{row['anchor_url']}\" {image_css}>\n")
            htmlfile.write("</td>\n")

            htmlfile.write("</tr>\n")
            
            # 3 context tweets after
            htmlfile.write("<tr>\n")

            # context11_url
            htmlfile.write("<td style=\"width:400; border-bottom: 1px solid black;\">\n")
            htmlfile.write(f"<img src=\"{row['context11_url']}\" {image_css}>\n")
            htmlfile.write("</td>\n")

            # context12_url
            htmlfile.write("<td style=\"width:400; border-bottom: 1px solid black;\">\n")
            htmlfile.write(f"<img src=\"{row['context12_url']}\" {image_css}>\n")
            htmlfile.write("</td>\n")

            # context13_url
            htmlfile.write("<td style=\"width:400; border-bottom: 1px solid black;\">\n")
            htmlfile.write(f"<img src=\"{row['context13_url']}\" {image_css}>\n")
            htmlfile.write("</td>\n")

            htmlfile.write("</tr>\n")
            
        htmlfile.write("</table>\n")

        # write the end
        htmlfile.write("</body>\n</html>")    

## Calculate the confusion matrix between the tweets with and without context

Load the tweets without context (only anchor tweets)

In [2]:
import os
import json

annot_nocontext_folder = 'data/annotations'
annot_nocontext_filename = 'full_gold.json'
annot_nocontext_filepath = os.path.join(annot_nocontext_folder, annot_nocontext_filename)

annot_nocontext = {}
with open(annot_nocontext_filepath, 'r') as jsonfile:
    lines = jsonfile.read().split("\n")[:-1]
    for line in lines:
        instance = json.loads(line)
        
        # add the instance_id so that two versions of annotation can be aligned
        instance_id = '_'.join(instance['anchor_id'].split("_")[:-1])
        
        # convert Other into No
        label = instance['adjudicated_label']
        label = label if label == 'Yes' else "No"
        instance['adjudicated_label'] = label
        
        instance['instance_id'] = instance_id
        annot_nocontext[instance_id] = instance

Load the tweets with context

In [ ]:
# replace the yes with no if confidence level is less than or equal to 4
import csv

annotation_csv = f'batches/2_finished.csv'
new_annot_csv = f'batches/new_2_finished.csv'
conf_threshold = 3.0
instances = []
with open(annotation_csv, 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        if int(row['Answer.Q2']) <= conf_threshold:
            row['Answer.Q1'] = 'No'
        instances.append(row)
        
with open(new_annot_csv, 'w') as csvfile:
    column_names = list(instances[0].keys())
    writer = csv.DictWriter(csvfile, fieldnames=column_names)
    writer.writeheader()
    for instance in instances:
        writer.writerow(instance)

In [13]:
annot_nocontext

{'springbreak2019_1106330449134174210': {'Answer.Q1_A1KXG3R48T74R3': 'Yes',
  'Answer.Q2_A1KXG3R48T74R3': '4',
  'adjudicated_label': 'Yes',
  'HITId': '39XCQ6V3L4SNFZ0V49SDK8POVHE65X',
  'image_filename': 'springbreak2019_anchor_1106330449134174210.jpg',
  'json_filename': 'springbreak2019_anchor_1106330449134174210.json',
  'anchor_location': 'Houston',
  'anchor_id': 'springbreak2019_1106330449134174210_1106330449134174210',
  'event': 'springbreak2019',
  'anchor_url': 'http://www.cse.unt.edu/~blanco/screenshots/anchor/springbreak2019_1106330449134174210_anchor_1106330449134174210.png',
  'instance_id': 'springbreak2019_1106330449134174210'},
 'veteransday2019_1193202077603905536': {'Answer.Q1_A1YO75Q6M618HU': 'Yes',
  'Answer.Q2_A1YO75Q6M618HU': '3',
  'adjudicated_label': 'Yes',
  'HITId': '3PKJ68EHETKBJNZXPUNBI9XJMWOJHW',
  'image_filename': 'veteransday2019_anchor_1193202077603905536.jpg',
  'json_filename': 'veteransday2019_anchor_1193202077603905536.json',
  'anchor_location'

Load the tweets with context (second batch)

In [5]:
# replace the yes with no if confidence level is less than or equal to 4
import csv

annotation_csv = f'batches/2_finished.csv'
new_annot_csv = f'batches/new_2_finished.csv'
conf_threshold = 3.0
instances = []
with open(annotation_csv, 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        if int(row['Answer.Q2']) <= conf_threshold:
            row['Answer.Q1'] = 'No'
        instances.append(row)
        
with open(new_annot_csv, 'w') as csvfile:
    column_names = list(instances[0].keys())
    writer = csv.DictWriter(csvfile, fieldnames=column_names)
    writer.writeheader()
    for instance in instances:
        writer.writerow(instance)

In [4]:
from annotation_helper import *
from collections import defaultdict, Counter

loc_tmp_dir = '/home/zhaomin/Documents/github/loc_tmp2'
annotation_csv = f'batches/new_2_finished.csv'

workers_removed = []
k_threshold = 0.6

# generate the file to use MACE, the generated file is 'annotation_list.csv'
annotation_list_csvfilepath = 'annotation_list.csv'
annotation_list, annotators, hitids = get_annotation_list(annotation_csv, workers_removed)
save_annotation_list(annotation_list, annotation_list_csvfilepath)

# run MACE
mace_prefix = f"check"
!java -jar MACE.jar --prefix $mace_prefix annotation_list.csv >/dev/null 2>&1

# load the competence file and prediction file
competence_file = f'{loc_tmp_dir}/{mace_prefix}.competence'
competence_annotators = read_competence(competence_file, annotators)
mace_prediction_file = f'{loc_tmp_dir}/{mace_prefix}.prediction'
predictions = read_prediction(mace_prediction_file)

raw_preds = {}
for index, hitid in enumerate(hitids):
    pred = predictions[index]
    raw_preds[hitid] = pred

max_num_instance = len([x[0] for x in annotation_list])

# get the workers_removed
print("-" * 60)
print("The Krippendorf's Alpha threshold is {}".format(k_threshold))
workers_removed = []
for index, score_annotator in enumerate(competence_annotators):
    rank = index + 1
    score, annotator = score_annotator[0], score_annotator[1]

    # calculate Krippendorf's Alpha
    annotation_list, annotators, hitids = get_annotation_list(annotation_csv, workers_removed)
    k_alpha = calculate_krippendorff(annotation_list)

    # if calculated alpha is lower than the threshold, then continue to remove the annotators
    if k_alpha <= k_threshold:
        workers_removed.append(annotator)
    else:
        print("The overall k_alpah is {:.2f}".format(k_alpha))
        break

print("You have removed {} annotators".format(len(workers_removed)))

# calculate the statistics
filtered_annotations_of_instance = read_annotation_of_instance(annotation_csv, workers_removed)
for index, hitid in enumerate(hitids):
    mace_prediction = predictions[index]
    if hitid in filtered_annotations_of_instance.keys():
        annots = filtered_annotations_of_instance[hitid]
        annots['adjudicated_label'] = mace_prediction
    else:
        continue

annot_context = {}

for _, annot in filtered_annotations_of_instance.items():
    instance_id = annot['Input.instance_id']
    annot_context[instance_id] = annot

Saving the annotation list for using the MACE...
Done.
------------------------------------------------------------
The Krippendorf's Alpha threshold is 0.6
The overall k_alpah is 0.65
You have removed 59 annotators


In [11]:
import csv

csv_filename = 'batches/2_finished.csv'
        
# generate the html file
i = 0
html_filename = "check_new_instances.html"
with open(html_filename, 'w') as htmlfile:

    with open(csv_filename, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        
        # write the start
        htmlfile.write("<!DOCTYPE html>\n<html>\n<head>\n<title>Tweet Example</title>\n</head>\n<body>\n")

        # use a big table to display the tweet example
        htmlfile.write("""<table style=\"margin-left: auto; margin-right: auto; border: 1px solid black; line-height: 1.0em; width: 1200\">\n""")
        image_css = """style=\"display: block; margin-left: auto; margin-right: auto; max-width: 450px; max-height: 650px;\""""
        
        for row in reader:

            if int(row['Answer.Q2']) > 3.0 or row['Answer.Q1'] == 'No':
                continue
            
            i += 1
            if i == 100:
                break
            
            # 3 context tweets before
            htmlfile.write("<tr>\n")

            # context8_url
            htmlfile.write("<td style=\"width:400; border-bottom: 1px solid black;\">\n")
            htmlfile.write(f"<img src=\"{row['Input.context8_url']}\" {image_css}>\n")
            htmlfile.write("</td>\n")

            # context9_url
            htmlfile.write("<td style=\"width:400; border-bottom: 1px solid black;\">\n")
            htmlfile.write(f"<img src=\"{row['Input.context9_url']}\" {image_css}>\n")
            htmlfile.write("</td>\n")

            # context10_url
            htmlfile.write("<td style=\"width:400; border-bottom: 1px solid black;\">\n")
            htmlfile.write(f"<img src=\"{row['Input.context10_url']}\" {image_css}>\n")
            htmlfile.write("</td>\n")

            htmlfile.write("</tr>\n")
            
            # anchor tweet
            htmlfile.write("<tr>\n")

            # anchor_url
            htmlfile.write("<td style=\"width:400; border-bottom: 1px solid black;\">\n")
            htmlfile.write(f"<img src=\"{row['Input.anchor_url']}\" {image_css}>\n")
            htmlfile.write("</td>\n")

            htmlfile.write("</tr>\n")
            
            # 3 context tweets after
            htmlfile.write("<tr>\n")

            # context11_url
            htmlfile.write("<td style=\"width:400; border-bottom: 1px solid black;\">\n")
            htmlfile.write(f"<img src=\"{row['Input.context11_url']}\" {image_css}>\n")
            htmlfile.write("</td>\n")

            # context12_url
            htmlfile.write("<td style=\"width:400; border-bottom: 1px solid black;\">\n")
            htmlfile.write(f"<img src=\"{row['Input.context12_url']}\" {image_css}>\n")
            htmlfile.write("</td>\n")

            # context13_url
            htmlfile.write("<td style=\"width:400; border-bottom: 1px solid black;\">\n")
            htmlfile.write(f"<img src=\"{row['Input.context13_url']}\" {image_css}>\n")
            htmlfile.write("</td>\n")

            htmlfile.write("</tr>\n")
            
        htmlfile.write("</table>\n")

        # write the end
        htmlfile.write("</body>\n</html>")    

In [3]:
import json
import os
import numpy as np

data_folder = 'data/annotations'
annot_filename = 'new_annot.json'
annot_filepath = os.path.join(data_folder, annot_filename)

i = 0
instances_labelchange = []
with open(annot_filepath) as annot_file:
    lines = annot_file.read().split("\n")[:-1]
    for line in lines:
        instance = json.loads(line)
        keys = [key for key in instance.keys() if key.startswith("Answer.Q1")]
        if len(keys) != 0:
            i += 1
            instance_id = instance['instance_id']
            label_nocontext = annot_nocontext[instance_id]['adjudicated_label']
            label_context = instance['adjudicated_label']
            if label_context != label_nocontext:
                instance['adjudicated_label_nocontext'] = label_nocontext
                instances_labelchange.append(instance)

In [60]:
labelchange_htmlfilename = 'label_change.html'

with open(labelchange_htmlfilename, 'w+') as htmlfile:
    
    # write the start
    htmlfile.write("<!DOCTYPE html>\n<html>\n<head>\n<title>Tweet Example</title>\n</head>\n<body>\n")

    # use a big table to display the tweet example
    htmlfile.write("""<table style=\"margin-left: auto; margin-right: auto; border: 1px solid black; line-height: 1.0em; width: 1200\">\n""")
    image_css = """style=\"display: block; margin-left: auto; margin-right: auto; max-width: 450px; max-height: 650px;\""""
    
    for instance in instances_labelchange:
        
        ###########################
        ### 3 context tweets before
        ###########################

        htmlfile.write("<tr>\n")

        # context8_url
        htmlfile.write("<td style=\"width:400; border-bottom: 1px solid black;\">\n")
        htmlfile.write(f"<img src=\"{instance['context8_url']}\" {image_css}>\n")
        htmlfile.write("</td>\n")

        # context9_url
        htmlfile.write("<td style=\"width:400; border-bottom: 1px solid black;\">\n")
        htmlfile.write(f"<img src=\"{instance['context9_url']}\" {image_css}>\n")
        htmlfile.write("</td>\n")

        # context10_url
        htmlfile.write("<td style=\"width:400; border-bottom: 1px solid black;\">\n")
        htmlfile.write(f"<img src=\"{instance['context10_url']}\" {image_css}>\n")
        htmlfile.write("</td>\n")

        htmlfile.write("</tr>\n")

        ###########################
        ### anchor tweet
        ###########################

        htmlfile.write("<tr>\n")
        htmlfile.write("<td style=\"width:400; border-bottom: 1px solid black;\">\n")
        htmlfile.write("<p style=\"text-align: center; font-weight: bold; font-size: large\">\n")
        htmlfile.write(f"Without context: {instance['adjudicated_label_nocontext']}<br><br>\n")
        htmlfile.write(f"With context: {instance['adjudicated_label']}<br><br>\n")
        htmlfile.write(f"Location: {instance['anchor_location']}<br><br>\n")
        htmlfile.write(f"instance_id: {instance['instance_id']}<br><br>\n")
        htmlfile.write("</p>\n")
        htmlfile.write("</td>\n")

        # anchor_url
        htmlfile.write("<td style=\"width:400; border-bottom: 1px solid black;\">\n")
        htmlfile.write(f"<img src=\"{instance['anchor_url']}\" {image_css}>\n")
        htmlfile.write("</td>\n")

        htmlfile.write("<td style=\"width:400; border-bottom: 1px solid black;\"></td>\n")
        htmlfile.write("</tr>\n")

        ###########################
        ### 3 context tweets after
        ###########################

        htmlfile.write("<tr>\n")

        # context11_url
        htmlfile.write("<td style=\"width:400; border-bottom: 1px solid black;\">\n")
        htmlfile.write(f"<img src=\"{instance['context11_url']}\" {image_css}>\n")
        htmlfile.write("</td>\n")

        # context12_url
        htmlfile.write("<td style=\"width:400; border-bottom: 1px solid black;\">\n")
        htmlfile.write(f"<img src=\"{instance['context12_url']}\" {image_css}>\n")
        htmlfile.write("</td>\n")

        # context13_url
        htmlfile.write("<td style=\"width:400; border-bottom: 1px solid black;\">\n")
        htmlfile.write(f"<img src=\"{instance['context13_url']}\" {image_css}>\n")
        htmlfile.write("</td>\n")

        htmlfile.write("</tr>\n")

        
    htmlfile.write("</table>\n")

    # write the end
    htmlfile.write("</body>\n</html>")    

In [46]:
from sklearn.metrics import confusion_matrix

confusion_matrix(labels_nocontext, labels_context, labels=["Yes", "No"])

array([[1314,  449],
       [1051,  680]], dtype=int64)

## get the examples of label change

In [11]:
nocontext_annot = 'data/annotations/full_gold.json'
context_annot = 'data/annotations/new_annot.json'

# load annotations without context
annot_nocontext = {}
with open(nocontext_annot, 'r') as jsonfile:
    lines = jsonfile.read().split("\n")[:-1]
    for line in lines:
        ins = json.loads(line)
        new_id = '_'.join(ins['anchor_id'].split("_")[:2])
        annot_nocontext[new_id] = ins
        
# load annotations with context, only keep the instances with changed label
instances = []
with open(context_annot, 'r') as jsonfile:
    lines = jsonfile.read().split("\n")[:-1]
    for line in lines:
        ins = json.loads(line)
        if annot_nocontext[ins['instance_id']]['adjudicated_label'] != ins['adjudicated_label']:
            instances.append(ins)

In [18]:
instances[0]

{'anchor_location': 'Atlanta',
 'instance_id': 'thanksgiving2019_1199840018380001281',
 'event': 'thanksgiving2019',
 'Answer.Q1_A2ZJS73XSSMRTD': 'Yes',
 'Answer.Q2_A2ZJS73XSSMRTD': '4',
 'adjudicated_label': 'Yes',
 'anchor_timestamp': 'Wed Nov 27 23:58:46 +0000 2019',
 'anchor_jsonpath': 'data/json_files/thanksgiving2019_1199840018380001281/anchor_1199840018380001281.json',
 'anchor_tweettext': 'Just a sampling of our Thanksgiving Eve feast. Special shout-out to @FOX5ATLCallaway and @Elepo for organizing it!!!!!! @FOX5Atlanta https://t.co/w6k2p0c46u',
 'anchor_url': 'http://www.cse.unt.edu/~blanco/screenshot/thanksgiving2019_1199840018380001281_anchor_1199840018380001281.png',
 'anchor_imagepath': 'data/image_files/thanksgiving2019_1199840018380001281/anchor_1199840018380001281.jpg',
 'context8_jsonpath': 'data/json_files/thanksgiving2019_1199840018380001281/1199503490923536389.json',
 'context8_tweettext': '@aungeliquefox5 Yes! And so generous !',
 'context8_url': 'http://www.cse.un

In [20]:
from IPython.display import display, Image

query_text = """6 Chris"""

for error in instances:
    texts = [value for key, value in error.items() if key.endswith("text")]
    for text in texts:
        if query_text in text:
            print(f"instance_id: {error['instance_id']}")
            print(f"Location: {error['anchor_location']}, gold_label: {error['adjudicated_label']}")
            
            print("-------------- Tweet 1 --------------\n")
            print(error['context8_tweettext'])
            display(Image(url=error['context8_url']))
            if 'context8_imagepath' in error.keys():
                display(Image(url=error['context8_imagepath']))
                
            print("-------------- Tweet 2 --------------\n")
            print(error['context9_tweettext'])
            display(Image(url=error['context9_url']))
            if 'context9_imagepath' in error.keys():
                display(Image(url=error['context9_imagepath']))
            
            print("-------------- Tweet 3 --------------\n")
            print(error['context10_tweettext'])
            display(Image(url=error['context10_url']))
            if 'context10_imagepath' in error.keys():
                display(Image(url=error['context10_imagepath']))
                
            print("-------------- Tweet 4 --------------\n")
            print(error['anchor_tweettext'])
            display(Image(url=error['anchor_url']))
            if 'anchor_imagepath' in error.keys():
                display(Image(url=error['anchor_imagepath']))
                
            print("-------------- Tweet 5 --------------\n")
            print(error['context11_tweettext'])
            display(Image(url=error['context11_url']))
            if 'context11_imagepath' in error.keys():
                display(Image(url=error['context11_imagepath']))
                
            print("-------------- Tweet 6 --------------\n")
            print(error['context12_tweettext'])
            display(Image(url=error['context12_url']))
            if 'context12_imagepath' in error.keys():
                display(Image(url=error['context12_imagepath']))
                
            print("-------------- Tweet 7 --------------\n")
            print(error['context13_tweettext'])
            display(Image(url=error['context13_url']))
            if 'context13_imagepath' in error.keys():
                display(Image(url=error['context13_imagepath']))
                
            break

instance_id: christmas2019_1214311533452378113
Location: Philadelphia, gold_label: Yes
-------------- Tweet 1 --------------

*watching a Michael Jackson documentary*
Journalist: “and later, the playground of evil!”
NOT THE PLAYGROUND OF EVIL!!! Lmao! 😂😂😂😂


-------------- Tweet 2 --------------

Sometimes it’s just as good as it gets LOL! https://t.co/4oD48JyQfm


-------------- Tweet 3 --------------

Get my 2020 calendar now at https://t.co/e2OCofUK1D https://t.co/RrYkpdSxU7


-------------- Tweet 4 --------------

6 Christmas tree recycling sites in Philadelphia open Mon-Fri., 8am-6pm. https://t.co/YvszmtV30r


-------------- Tweet 5 --------------

Prayers have never put out a fire or helped relief reach those that need it. Please donate anything you can! https://t.co/j2BdKj16xs #Australia #RedCross #Donate #PrayersForAustrailia https://t.co/D6A9yzt3jm


-------------- Tweet 6 --------------

And in South Philly, right outside my bedroom window, a big beautiful hawk! ❤️💛🤍🖤 https://t.co/VPcEwr4fxs


-------------- Tweet 7 --------------

Increased my treadmill speed today n my legs r killin me https://t.co/u8SFIZ6VMp
